In [1]:
import pandas as pd
import numpy as np
import aux.perfil as pf
import aux.acessos as ac

## Janeiro 2021 (ano anterior)

In [2]:
# ###################### JANEIRO DE 2021 ######################

query_aa = """

select * from 
(with ativos_pag as (
select distinct 
pla_cd_conta_cartao
FROM processed_zone_database_oracle_pag.vw_autorizacao a 
left join processed_zone_database_oracle_pag.plastico p
on p.pla_cd_plastico = a.aut_cd_plastico
WHERE aut_cd_unidade_credenciada <> 60024
AND aut_fl_saque = 'N'
AND aut_fl_limite_utilizado = 'C'
AND aut_nr_randomico_cancelamento = 0
AND date_trunc ('month', aut_dt_autorizacao) = to_date('2021-01-01', 'yyyy-mm-dd') 
)
, ativados_pag as (
select distinct 
pla_cd_conta_cartao
FROM processed_zone_database_oracle_pag.vw_autorizacao a 
left join processed_zone_database_oracle_pag.plastico p
on p.pla_cd_plastico = a.aut_cd_plastico
WHERE aut_cd_unidade_credenciada <> 60024
AND aut_fl_saque = 'N'
AND aut_fl_limite_utilizado = 'C'
AND aut_nr_randomico_cancelamento = 0
AND aut_dt_autorizacao <= to_date('2021-01-31', 'yyyy-mm-dd') 
)
, atraso_max as 
(
	select
	max(dt_snapshot_date) AS max_registro
	from platform_curated_zone.cre_daily_debtors 
	where date_trunc('month', cast(dt_snapshot_date as date)) = to_date('2021-01-01', 'yyyy-mm-dd') 
)
, base_atraso as (
	select  * 
	from platform_curated_zone.cre_daily_debtors  acd
	inner join atraso_max mat on mat.max_registro = acd.dt_snapshot_date 
)
, inad_pag as (
select distinct coc.coc_cd_conta_cartao	from processed_zone_database_oracle_pag.conta_cartao coc
inner join base_atraso atr on coc.coc_cd_conta_cartao = atr.id_account_number
where (coc_cd_tipo_cartao between 2624 and 2628
or coc_cd_tipo_cartao between 2630 and 2632)
and date_trunc('month', coc.coc_dt_abertura_conta) = to_date('2021-01-01',  'yyyy-mm-dd')
and atr.nr_dias_atraso > 30
)
select
coc.coc_cd_conta_cartao as conta
, cli.cli_nr_cpf as cpf,
round((extract(day from to_date('2021-01-01',  'yyyy-mm-dd') - cast(coc_dt_Abertura_conta as date))/30.5/12),0) as tempo_conta,
'pag' as negocio
from processed_zone_database_oracle_pag.conta_cartao coc
inner join processed_zone_database_oracle_pag.cliente cli on (cli.cli_cd_cliente = coc.coc_cd_cliente)
left join ativos_pag ap on (ap.pla_cd_conta_cartao = coc.coc_cd_conta_cartao)
inner join ativados_pag atp on (atp.pla_cd_conta_cartao = coc.coc_cd_conta_cartao) --já ativou
left join inad_pag i on (i.coc_cd_conta_cartao = coc.coc_cd_conta_cartao)
where i.coc_cd_conta_cartao is null --adimplente
and ap.pla_cd_conta_cartao is null --não ativo nesse mês
and coc_fl_situacao = 1
and coc_nr_conta_cfi is not null
and coc_dt_abertura_conta < to_date('2021-01-01',  'yyyy-mm-dd')
)
union all 
select * from (
---------------------------------
---------QUERY WILL -------------
---------------------------------
with ativos_will as (
select distinct aut.cpf
from "platform_curated_zone".unique_authorizations_full aut					
inner join					
(select cast(substring(created_at,1,10) as date) as created_at, 
cast(fast_account as bigint) nrconta 
from "processed-zone-database-customer".customer_card_account) cca					
on aut.numerodaconta = cca.nrconta					
where cast(cca.created_at as date) <=  cast(aut.dt_autorizacao as date)					
and aut.status_compra = 'aprovada' and aut.codretorno = '00' and aut.mti in(100,120,200)
and date_trunc ('month', dt_autorizacao) = to_date('2021-01-01', 'yyyy-mm-dd') 
)
, ativados_will as (
select distinct aut.cpf
from "platform_curated_zone".unique_authorizations_full aut					
inner join					
(select substring(created_at, 1,10) as created_at, 
cast(fast_account as bigint) nrconta 
from "processed-zone-database-customer".customer_card_account) cca					
on aut.numerodaconta = cca.nrconta					
where cast(cca.created_at as date) <= cast(aut.dt_autorizacao as date)								
and aut.status_compra = 'aprovada' and aut.codretorno = '00' and aut.mti in(100,120,200)
and dt_autorizacao <= to_date('2021-01-31', 'yyyy-mm-dd') 
)
, inad_will as (select distinct id_customer from
(
select  *from "platform_curated_zone".customer_daily_view acd
inner join
(select
max(dt_snapshot_date) AS max_registro
from "platform_curated_zone".customer_daily_view
where date_trunc ('month', dt_snapshot_date)  = to_date('2021-01-01', 'yyyy-mm-dd') 
) mat on mat.max_registro = acd.dt_snapshot_date
) where  nr_days_paste_due > 30
)
select
0 as conta, 
p.cd_cpf as cpf,
round((extract(day from to_date('2021-01-01',  'yyyy-mm-dd') - cast(dt_created_user as date))/30.5/12),0) as tempo_conta,
'will' as negocio
from growth_curated_zone.proposal_general p
inner join growth_curated_zone.clientes c on (p.cd_cpf = c.cpf)
left join ativos_will ap on (ap.cpf = p.cd_cpf)
inner join ativados_will atp on (atp.cpf = p.cd_cpf) --já ativou
left join inad_will i on (i.id_customer = c.id_customer)
where i.id_customer is null --adimplente
and ap.cpf is null --não ativo nesse mês
and p.ds_origin = 'will'
and dt_created_user < to_date('2021-01-01',  'yyyy-mm-dd')
)



"""

df_aa = ac.df_athena('flavia-costa', query_aa)

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [3]:
len(df_aa)

576223

## Ano atual (Janeiro de 2022)

In [4]:
# ###################### JANEIRO DE 2022 ######################

query_at = """

select * from 
(with ativos_pag as (
select distinct 
pla_cd_conta_cartao
FROM processed_zone_database_oracle_pag.vw_autorizacao a 
left join processed_zone_database_oracle_pag.plastico p
on p.pla_cd_plastico = a.aut_cd_plastico
WHERE aut_cd_unidade_credenciada <> 60024
AND aut_fl_saque = 'N'
AND aut_fl_limite_utilizado = 'C'
AND aut_nr_randomico_cancelamento = 0
AND date_trunc ('month', aut_dt_autorizacao) = to_date('2022-01-01', 'yyyy-mm-dd') 
)
, ativados_pag as (
select distinct 
pla_cd_conta_cartao
FROM processed_zone_database_oracle_pag.vw_autorizacao a 
left join processed_zone_database_oracle_pag.plastico p
on p.pla_cd_plastico = a.aut_cd_plastico
WHERE aut_cd_unidade_credenciada <> 60024
AND aut_fl_saque = 'N'
AND aut_fl_limite_utilizado = 'C'
AND aut_nr_randomico_cancelamento = 0
AND aut_dt_autorizacao <= to_date('2022-01-31', 'yyyy-mm-dd') 
)
, atraso_max as 
(
	select
	max(dt_snapshot_date) AS max_registro
	from platform_curated_zone.cre_daily_debtors 
	where date_trunc('month', cast(dt_snapshot_date as date)) = to_date('2022-01-01', 'yyyy-mm-dd') 
)
, base_atraso as (
	select  * 
	from platform_curated_zone.cre_daily_debtors  acd
	inner join atraso_max mat on mat.max_registro = acd.dt_snapshot_date 
)
, inad_pag as (
select distinct coc.coc_cd_conta_cartao	from processed_zone_database_oracle_pag.conta_cartao coc
inner join base_atraso atr on coc.coc_cd_conta_cartao = atr.id_account_number
where (coc_cd_tipo_cartao between 2624 and 2628
or coc_cd_tipo_cartao between 2630 and 2632)
and date_trunc('month', coc.coc_dt_abertura_conta) = to_date('2022-01-01',  'yyyy-mm-dd')
and atr.nr_dias_atraso > 30
)
select
coc.coc_cd_conta_cartao as conta
, cli.cli_nr_cpf as cpf,
round((extract(day from to_date('2022-01-01',  'yyyy-mm-dd') - cast(coc_dt_abertura_conta as date))/30.5/12),0) as tempo_conta,
'pag' as negocio
from processed_zone_database_oracle_pag.conta_cartao coc
inner join processed_zone_database_oracle_pag.cliente cli on (cli.cli_cd_cliente = coc.coc_cd_cliente)
left join ativos_pag ap on (ap.pla_cd_conta_cartao = coc.coc_cd_conta_cartao)
inner join ativados_pag atp on (atp.pla_cd_conta_cartao = coc.coc_cd_conta_cartao) --já ativou
left join inad_pag i on (i.coc_cd_conta_cartao = coc.coc_cd_conta_cartao)
where i.coc_cd_conta_cartao is null --adimplente
and ap.pla_cd_conta_cartao is null --não ativo nesse mês
and coc_fl_situacao = 1
and coc_nr_conta_cfi is not null
and coc_dt_abertura_conta < to_date('2022-01-01',  'yyyy-mm-dd')
)
union all 
select * from (
---------------------------------
---------QUERY WILL -------------
---------------------------------
with ativos_will as (
select distinct aut.cpf
from "platform_curated_zone".unique_authorizations_full aut					
inner join					
(select cast(substring(created_at,1,10) as date) as created_at, 
cast(fast_account as bigint) nrconta 
from "processed-zone-database-customer".customer_card_account) cca					
on aut.numerodaconta = cca.nrconta					
where cast(cca.created_at as date) <=  cast(aut.dt_autorizacao as date)					
and aut.status_compra = 'aprovada' and aut.codretorno = '00' and aut.mti in(100,120,200)
and date_trunc ('month', dt_autorizacao) = to_date('2022-01-01', 'yyyy-mm-dd') 
)
, ativados_will as (
select distinct aut.cpf
from "platform_curated_zone".unique_authorizations_full aut					
inner join					
(select substring(created_at, 1,10) as created_at, 
cast(fast_account as bigint) nrconta 
from "processed-zone-database-customer".customer_card_account) cca					
on aut.numerodaconta = cca.nrconta					
where cast(cca.created_at as date) <= cast(aut.dt_autorizacao as date)								
and aut.status_compra = 'aprovada' and aut.codretorno = '00' and aut.mti in(100,120,200)
and dt_autorizacao <= to_date('2022-01-31', 'yyyy-mm-dd') 
)
, inad_will as (select distinct id_customer from
(
select  *from "platform_curated_zone".customer_daily_view acd
inner join
(select
max(dt_snapshot_date) AS max_registro
from "platform_curated_zone".customer_daily_view
where date_trunc ('month', dt_snapshot_date)  = to_date('2022-01-01', 'yyyy-mm-dd') 
) mat on mat.max_registro = acd.dt_snapshot_date
) where  nr_days_paste_due > 30
)
select
0 as conta, 
p.cd_cpf as cpf,
round((extract(day from to_date('2022-01-01',  'yyyy-mm-dd') - cast(dt_created_user as date))/30.5/12),0) as tempo_conta,
'will' as negocio
from growth_curated_zone.proposal_general p
inner join growth_curated_zone.clientes c on (p.cd_cpf = c.cpf)
left join ativos_will ap on (ap.cpf = p.cd_cpf)
inner join ativados_will atp on (atp.cpf = p.cd_cpf) --já ativou
left join inad_will i on (i.id_customer = c.id_customer)
where i.id_customer is null --adimplente
and ap.cpf is null --não ativo nesse mês
and p.ds_origin = 'will'
and dt_created_user < to_date('2022-01-01',  'yyyy-mm-dd')
)


"""

df_at = ac.df_athena('flavia-costa', query_at)

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [5]:
len(df_at)

949259

## Mês anterior (Dez/19)

In [6]:
# ###################### DEZ DE 2021 ######################

query_ma = """

select * from 
(with ativos_pag as (
select distinct 
pla_cd_conta_cartao
FROM processed_zone_database_oracle_pag.vw_autorizacao a 
left join processed_zone_database_oracle_pag.plastico p
on p.pla_cd_plastico = a.aut_cd_plastico
WHERE aut_cd_unidade_credenciada <> 60024
AND aut_fl_saque = 'N'
AND aut_fl_limite_utilizado = 'C'
AND aut_nr_randomico_cancelamento = 0
AND date_trunc ('month', aut_dt_autorizacao) = to_date('2021-12-01', 'yyyy-mm-dd') 
)
, ativados_pag as (
select distinct 
pla_cd_conta_cartao
FROM processed_zone_database_oracle_pag.vw_autorizacao a 
left join processed_zone_database_oracle_pag.plastico p
on p.pla_cd_plastico = a.aut_cd_plastico
WHERE aut_cd_unidade_credenciada <> 60024
AND aut_fl_saque = 'N'
AND aut_fl_limite_utilizado = 'C'
AND aut_nr_randomico_cancelamento = 0
AND aut_dt_autorizacao <= to_date('2021-12-31', 'yyyy-mm-dd') 
)
, atraso_max as 
(
	select
	max(dt_snapshot_date) AS max_registro
	from platform_curated_zone.cre_daily_debtors 
	where date_trunc('month', cast(dt_snapshot_date as date)) = to_date('2021-12-01', 'yyyy-mm-dd') 
)
, base_atraso as (
	select  * 
	from platform_curated_zone.cre_daily_debtors  acd
	inner join atraso_max mat on mat.max_registro = acd.dt_snapshot_date 
)
, inad_pag as (
select distinct coc.coc_cd_conta_cartao	from processed_zone_database_oracle_pag.conta_cartao coc
inner join base_atraso atr on coc.coc_cd_conta_cartao = atr.id_account_number
where (coc_cd_tipo_cartao between 2624 and 2628
or coc_cd_tipo_cartao between 2630 and 2632)
and date_trunc('month', coc.coc_dt_abertura_conta) = to_date('2021-12-01',  'yyyy-mm-dd')
and atr.nr_dias_atraso > 30
)
select
coc.coc_cd_conta_cartao as conta
, cli.cli_nr_cpf as cpf,
round((extract(day from to_date('2021-12-01',  'yyyy-mm-dd') - cast(coc_dt_abertura_conta as date))/30.5/12),0) as tempo_conta,
'pag' as negocio
from processed_zone_database_oracle_pag.conta_cartao coc
inner join processed_zone_database_oracle_pag.cliente cli on (cli.cli_cd_cliente = coc.coc_cd_cliente)
left join ativos_pag ap on (ap.pla_cd_conta_cartao = coc.coc_cd_conta_cartao)
inner join ativados_pag atp on (atp.pla_cd_conta_cartao = coc.coc_cd_conta_cartao) --já ativou
left join inad_pag i on (i.coc_cd_conta_cartao = coc.coc_cd_conta_cartao)
where i.coc_cd_conta_cartao is null --adimplente
and ap.pla_cd_conta_cartao is null --não ativo nesse mês
and coc_fl_situacao = 1
and coc_nr_conta_cfi is not null
and coc_dt_abertura_conta < to_date('2021-12-01',  'yyyy-mm-dd')
)
union all 
select * from (
---------------------------------
---------QUERY WILL -------------
---------------------------------
with ativos_will as (
select distinct aut.cpf
from "platform_curated_zone".unique_authorizations_full aut					
inner join					
(select cast(substring(created_at,1,10) as date) as created_at, 
cast(fast_account as bigint) nrconta 
from "processed-zone-database-customer".customer_card_account) cca					
on aut.numerodaconta = cca.nrconta					
where cast(cca.created_at as date) <=  cast(aut.dt_autorizacao as date)					
and aut.status_compra = 'aprovada' and aut.codretorno = '00' and aut.mti in(100,120,200)
and date_trunc ('month', dt_autorizacao) = to_date('2021-12-01', 'yyyy-mm-dd') 
)
, ativados_will as (
select distinct aut.cpf
from "platform_curated_zone".unique_authorizations_full aut					
inner join					
(select substring(created_at, 1,10) as created_at, 
cast(fast_account as bigint) nrconta 
from "processed-zone-database-customer".customer_card_account) cca					
on aut.numerodaconta = cca.nrconta					
where cast(cca.created_at as date) <= cast(aut.dt_autorizacao as date)								
and aut.status_compra = 'aprovada' and aut.codretorno = '00' and aut.mti in(100,120,200)
and dt_autorizacao <= to_date('2021-12-31', 'yyyy-mm-dd') 
)
, inad_will as (select distinct id_customer from
(
select  *from "platform_curated_zone".customer_daily_view acd
inner join
(select
max(dt_snapshot_date) AS max_registro
from "platform_curated_zone".customer_daily_view
where date_trunc ('month', dt_snapshot_date)  = to_date('2021-12-01', 'yyyy-mm-dd') 
) mat on mat.max_registro = acd.dt_snapshot_date
) where  nr_days_paste_due > 30
)
select
0 as conta, 
p.cd_cpf as cpf,
round((extract(day from to_date('2021-12-01',  'yyyy-mm-dd') - cast(dt_created_user as date))/30.5/12),0) as tempo_conta,
'will' as negocio
from growth_curated_zone.proposal_general p
inner join growth_curated_zone.clientes c on (p.cd_cpf = c.cpf)
left join ativos_will ap on (ap.cpf = p.cd_cpf)
inner join ativados_will atp on (atp.cpf = p.cd_cpf) --já ativou
left join inad_will i on (i.id_customer = c.id_customer)
where i.id_customer is null --adimplente
and ap.cpf is null --não ativo nesse mês
and p.ds_origin = 'will'
and dt_created_user < to_date('2021-12-01',  'yyyy-mm-dd')
)


"""

df_ma = ac.df_athena('flavia-costa', query_ma)

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [7]:
len(df_ma)

878175

## Ativos Mês atual

In [8]:
query_atv = """

select * from 
(
with ativos_pag as (
select distinct 
pla_cd_conta_cartao
FROM processed_zone_database_oracle_pag.vw_autorizacao a 
left join processed_zone_database_oracle_pag.plastico p
on p.pla_cd_plastico = a.aut_cd_plastico
WHERE aut_cd_unidade_credenciada <> 60024
AND aut_fl_saque = 'N'
AND aut_fl_limite_utilizado = 'C'
AND aut_nr_randomico_cancelamento = 0
AND date_trunc ('month', aut_dt_autorizacao) = to_date('2022-01-01', 'yyyy-mm-dd') 
)
select
coc.coc_cd_conta_cartao as conta
, cli.cli_nr_cpf as cpf,
round((extract(day from to_date('2022-01-01',  'yyyy-mm-dd') - cast(coc_dt_abertura_conta as date))/30.5/12),0) as tempo_conta,  
'pag' as negocio
from processed_zone_database_oracle_pag.conta_cartao coc
inner join processed_zone_database_oracle_pag.cliente cli on (cli.cli_cd_cliente = coc.coc_cd_cliente)
left join ativos_pag ap on (ap.pla_cd_conta_cartao = coc.coc_cd_conta_cartao)
WHERE ap.pla_cd_conta_cartao is not null --ativo nesse mês
and coc_fl_situacao = 1
and coc_nr_conta_cfi is not null
and coc_dt_abertura_conta < to_date('2022-01-01',  'yyyy-mm-dd')
)
union all 
select * from (
with ativos_will as (
select distinct aut.cpf
from "platform_curated_zone".unique_authorizations_full aut					
inner join					
(select cast(substring(created_at,1,10) as date) as created_at, 
cast(fast_account as bigint) nrconta 
from "processed-zone-database-customer".customer_card_account) cca					
on aut.numerodaconta = cca.nrconta					
where cast(cca.created_at as date) <=  cast(aut.dt_autorizacao as date)					
and aut.status_compra = 'aprovada' and aut.codretorno = '00' and aut.mti in(100,120,200)
and date_trunc ('month', dt_autorizacao) = to_date('2022-01-01', 'yyyy-mm-dd') 
)
select
0 as conta, 
p.cd_cpf as cpf,
round((extract(day from to_date('2022-01-01',  'yyyy-mm-dd') - cast(dt_created_user as date))/30.5/12),0) as tempo_conta,  
'will' as negocio
from growth_curated_zone.proposal_general p
inner join growth_curated_zone.clientes c on (p.cd_cpf = c.cpf)
left join ativos_will ap on (ap.cpf = p.cd_cpf)
where ap.cpf is not null --ativo nesse mês
and p.ds_origin = 'will'
and dt_created_user < to_date('2022-01-01',  'yyyy-mm-dd')
)

"""

df_atv = ac.df_athena('flavia-costa', query_atv)

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [9]:
len(df_atv)

1035376

In [10]:
df_aa['periodo'] = '1 - churn jan-21'
df_ma['periodo'] = '2 - churn dez-21'
df_at['periodo'] = '3 - churn jan-22'
df_atv['periodo'] = '4 - ativos jan-22'

In [11]:
df_total = df_aa.append(df_ma).append(df_at).append(df_atv)

df_total['cont'] = 1

<ipython-input-11-c1fcd22c36f7>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total = df_aa.append(df_ma).append(df_at).append(df_atv)
<ipython-input-11-c1fcd22c36f7>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total = df_aa.append(df_ma).append(df_at).append(df_atv)


In [ ]:
path_to_csv = 'data_science/team/Flavia'
filename = 'base_churn.csv'
delim = ';'

ac.save_to_s3(path_to_csv, filename, df_total, delim)

In [12]:
df_total = pf.traz_info('flavia-costa', df_total, 'cpf')

Failed to execute query.
Traceback (most recent call last):
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/common.py", line 307, in _execute
    query_id = retry_api_call(
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 430, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 367, in iter
    return fut.result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 433, in result
    return self.__get_result()
  File "/usr/lib/python3.9/concurrent/futures/_base.py", line 389, in __get_result
    raise self._exception
  File "/home/flavia.costa/.local/lib/python3.9/site-packages/tenacity/__init__.py", line 433, in __call__
    result = fn(*args, **kwargs)
  File "/home/flavia.

In [13]:
len(df_total)

3446535

In [14]:
def tempo_conta(x):
    if x <= 0:
        return 'a. ate 1 ano de conta'
    elif x == 1:
        return 'b. 1-2 anos de conta'
    elif x == 2:
        return 'c. 2-3 anos de conta'
    elif x == 3:
        return 'd. 3-4 anos de conta'
    elif x == 4:
        return 'e. 4-5 anos de conta'
    elif x == 5:
        return 'f. 5-6 anos de conta'   
    else:
        return 'g. sem tempo'

In [15]:
df_total['pf_idade_conta'] = df_total['tempo_conta'].apply(tempo_conta)

In [16]:
df_total.columns

Index(['conta', 'cpf', 'tempo_conta', 'negocio', 'periodo', 'cont', 'chave',
       'chave_or', 'ds_origin', 'pf_dt_nascimento', 'base_perfil', 'pf_genero',
       'pf_escolaridade', 'pf_renda_declarada_will', 'pf_profissao',
       'pf_estado_civil', 'pf_estado', 'pf_regiao', 'pf_idade_conta',
       'pf_faixa_idade', 'pf_cidade', 'pf_top_mcc', 'pf_cep', 'pf_recencia',
       'pf_fx_recencia_mes', 'vl_real_hist', 'transacoes_hist',
       'spending90dias', 'transacoes90dias', 'spending_ultimo_ano',
       'transacoes_ultimo_ano'],
      dtype='object')

In [17]:
# df_agg = pf.agg_perfil_grupos(df_total, 'periodo')
campos_perfil = ['pf_genero', 'pf_faixa_idade', 'pf_estado', 'pf_regiao', 'pf_estado_civil', 'pf_escolaridade', 'pf_idade_conta']

#'pf_renda_declarada_will', 'pf_top_mcc'

soma = ['cont']

var_grupo = ['periodo']

chave2 = campos_perfil + soma + var_grupo

campo_agg = campos_perfil + var_grupo

agg = df_total[chave2].groupby(campo_agg, as_index=False).agg({'cont':['sum']})


In [18]:
agg['cont'].sum()

sum    3431889
dtype: int64

In [19]:
len(agg)

151081

In [20]:
agg.to_csv('agg_churn_fev22.csv')

In [21]:
df_pf = df_total[['pf_genero', 'pf_faixa_idade', 'pf_estado', 'pf_regiao', 'pf_estado_civil', 'pf_profissao', 'pf_escolaridade', 'pf_idade_conta', 'pf_renda_declarada_will', 'pf_top_mcc', 'periodo', 'cont']]

In [22]:
pf.report_perfil_grupos(df_pf.query('periodo == "4 - ativos jan-22" or periodo == "3 - churn jan-22"'), 'periodo', 'cont', 0.05, 50)

/home/flavia.costa/work/git/codigos/aux/perfil.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna_contagem] = df[coluna_contagem].astype('int')
/home/flavia.costa/work/git/codigos/aux/perfil.py:317: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  casos_signif_final = casos_signif_final.append(casos_signif).sort_values('diferenca_abs', ascending = False)
/home/flavia.costa/work/git/codigos/aux/perfil.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

report dos perfis finalizado!


/home/flavia.costa/work/git/codigos/aux/perfil.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  casos_signif_final = casos_signif_final.append(casos_signif).sort_values('diferenca_abs', ascending = False)


report dos perfis finalizado!


/home/flavia.costa/work/git/codigos/aux/perfil.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  casos_signif_final = casos_signif_final.append(casos_signif).sort_values('diferenca_abs', ascending = False)


...

/home/flavia.costa/work/git/codigos/aux/perfil.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  casos_signif_final = casos_signif_final.append(casos_signif).sort_values('diferenca_abs', ascending = False)


report dos perfis finalizado!


/home/flavia.costa/work/git/codigos/aux/perfil.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  casos_signif_final = casos_signif_final.append(casos_signif).sort_values('diferenca_abs', ascending = False)


report dos perfis finalizado!
report dos perfis finalizado!


/home/flavia.costa/work/git/codigos/aux/perfil.py:368: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  casos_signif_final = casos_signif_final.append(casos_signif).sort_values('diferenca_abs', ascending = False)


In [23]:
df_pag = df_total.query('negocio == "pag"')
df_will = df_total.query('negocio == "will"')

In [24]:
pf.agg_perfil_grupos(df_pag, 'periodo').to_csv('agg_churn_fev22_pag.csv')

In [25]:
pf.agg_perfil_grupos(df_will, 'periodo').to_csv('agg_churn_fev22_will.csv')

In [26]:
df_total[['periodo', 'pf_genero', 'cont']].groupby(['periodo', 'pf_genero']).sum()

cont
periodo           pf_genero        
1 - churn jan-21              15663
                  FEMALE     283508
                  MALE       277529
2 - churn dez-21              24151
                  FEMALE     434173
                  MALE       421129
3 - churn jan-22              25564
                  FEMALE     473730
                  MALE       451369
4 - ativos jan-22             19300
                  FEMALE     564012
                  MALE       456176

In [ ]:
pf.graf_catplot(df_total, 'periodo')